In [12]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("D:\\KHDL\\Phân loại mục đích sử dụng và phân cụm laptop\\crawl_data\\raw_data\\mega.csv")  # đổi tên nếu cần
print("Kích thước dữ liệu:", df.shape)

Kích thước dữ liệu: (919, 176)


In [13]:
import unidecode
import re
# Loại bỏ dấu, chuyển về chữ thường, thay khoảng trắng thành "_", loại bỏ ký tự đặc biệt
df.columns = [
    re.sub(r'[^a-z0-9_]', '', unidecode.unidecode(col).strip().lower().replace(" ", "_").replace("__", "_"))
    for col in df.columns
]

print("Số cột sau chuẩn hóa:", len(df.columns))

Số cột sau chuẩn hóa: 176


In [14]:
# Kiểm tra tên cột gốc
original_columns = df.columns.tolist()

# Loại bỏ các cột trùng tên (giữ cột đầu tiên)
seen = set()
new_columns = []
drop_indices = []

for idx, col in enumerate(original_columns):
    if col not in seen:
        seen.add(col)
        new_columns.append(col)
    else:
        drop_indices.append(idx)

# Số cột bị loại bỏ
num_removed_columns = len(drop_indices)
print(f"Số lượng cột bị loại bỏ do trùng tên: {num_removed_columns}")

# Xóa các cột trùng tên theo chỉ số (giữ lại cột đầu tiên)
df = df.loc[:, ~df.columns.duplicated(keep='first')]


Số lượng cột bị loại bỏ do trùng tên: 39


In [15]:
df.columns

Index(['product_name', 'price', 'san_pham', 'ten_hang', 'model', 'bo_vi_xu_ly',
       'card_do_hoa', 'bo_nho_ram', 'o_cung', 'so_cong_luu_tru_toi_da',
       ...
       'o_quang', 'unnamed_161', 'keyboard', 'khe_mo_rong',
       'kieu_khe_m2_ho_tr', 'may_tinh_xach_tay', 'tinh_nang_khac',
       'phu_kien_kem', 'bluetoothr', 'other_supports'],
      dtype='object', length=137)

In [16]:
df.rename(columns={'kich_thuoc': 'Độ Dày'}, inplace=True)

df.rename(columns={'trong_luong': 'Trọng Lượng'}, inplace=True)

df.rename(columns={'ten_hang': 'Hãng Sản Xuất'}, inplace=True)


In [17]:
def combine_columns(df, new_column_name, columns_to_merge):
    existing = [col for col in columns_to_merge if col in df.columns]
    if existing:
        df[new_column_name] = df[existing].bfill(axis=1).iloc[:, 0]
        df.drop(columns=[col for col in existing if col != new_column_name], inplace=True)
    return df

df = combine_columns(df,'Hãng Sản Xuất',[
    'Hãng Sản Xuất','thuong_hieu','hang_san_xuat'
])
df = combine_columns(df,'Độ Dày',[
    'Độ Dày','kich_thuoc_dai_x_rong_x_cao'
])
df = combine_columns(df, 'CPU', [''
    'cpu', 'bo_vi_xu_ly', 'bo_xu_ly_-_cpu',
    'bo_vxl', 'bo_xu_ly', 'cong_nghe_cpu',
    'cpu','bo_xu_ly__cpu'
])
df = combine_columns(df, 'RAM', [
    'ram', 'bo_nho_ram','dung_luong_ram',
    'ram','bo_nho_trong__ram'
])
df = combine_columns(df, 'Bộ Nhớ', [
    'o_cung', 'storage','o_dia_cung',
    'luu_tru', 'bo_nho_trong', 'o_cung/_o_dia_quang',
    'bo_nho_trong_-_ram', 'bo_nho', 'storage','dung_luong'
])
# Nhóm HDD
df = combine_columns(df, 'HDD', [
    'o_dia_cung_-_hdd','o_dia_cung__hdd','o_dia_cung_hdd'
])
# Nhóm SSD
df = combine_columns(df, 'SSD', [
    'ssd', 'o_dia_cung_-_ssd','o_dia_cung__ssd'
])

# Nhóm GPU
df = combine_columns(df, 'GPU', [
    'vga', 'card_do_hoa', 'gpu','card_man_hinh', 'card_do_hoa_-_video', 'loai_card_do_hoa', 'chip_do_hoa', 'card_vga', 'cac_do_hoa','do_hoa','card_do_hoa__video'
])

# Nhóm Battery
df = combine_columns(df, 'Dung Lượng Pin', [
    'pin', 'battery','pin_(battery)', 'pin(battery)', 'thoi_luong_pin', 'kieu_pin','pin_battery','pinbattery'
])

# Nhóm OS
df = combine_columns(df, 'Hệ Điều Hành', [
    'he_dieu_hanh', 'os','he_dieu_hanh_(ban_quyen)_di_kem', 'he_dieu_hanh_tuong_thich'
])

# Nhóm WiFi
df = combine_columns(df, 'Mạng WiFi', [
    'wifi', 'wireless', 'wifi,_bluetooth', 'wi-fi,_bluetooth',
    'wi-fi', 'wi-fi,', 'chuan_wifi', 'chuan_wi-fi','wifi_bluetooth','wifi,_bluetooth', 'wi-fi,_bluetooth'
])

# Nhóm Bluetooth
df = combine_columns(df, 'Bluetooth', [
    'bluetooth','bluetooth(r)', 'bluetooth','bluetoothr'
])

# Nhóm Audio
df = combine_columns(df, 'Âm Thanh', [
    'audio', 'am_thanh', 'cong_nghe_am_thanh'
])

# Nhóm Security
df = combine_columns(df, 'Bảo Mật', [
    'bao_mat', 'bao_mat_van_tay', 'nhan_dang_van_tay', 'van_tay', 'finger_print'
])

# Nhóm Card Reader
df = combine_columns(df, 'Đọc Thẻ', [
    'doc_the', 'doc_the_nho', 'doc_the_-_card_reader', 'khe_doc_the_nho', 'card_reader','doc_the__card_reader', 'the_doc', 'doc_the_nho'
])

# Nhóm LAN
df = combine_columns(df, 'Mạng LAN', [
    'lan', 'chuan_lan', 'ket_noi_co_day_(lan)','ket_noi_mang_lan', 'mang', 'thong_so_(lan/wireless)','thong_so_lanwireless','ket_noi_co_day_lan'
])

# Nhóm Keyboard
df = combine_columns(df, 'Bàn Phím', [
    'ban_phim', 'keyboard', 'keyboard_+_mouse','kieu_ban_phim', 'keyboard','keyboard__mouse'
])

# Nhóm Color
df = combine_columns(df, 'Màu Sắc', [
    'mau_sac', 'mau_sac/_chat_lieu', 'color','mau', 'mau_sac_(colour)','mau_sac_chat_lieu','mau_sac_colour'
])

# Nhóm Material
df = combine_columns(df, 'Chất Liệu', [
    'chat_lieu', 'chat_lieu_san_pham', 'thiet_ke/_chat_lieu_vo'
])

# Nhóm Ports
df = combine_columns(df, 'Cổng Giao Tiếp', [
    'cong_giao_tiep', 'cong_giao_tiep_-_port', 'cong_i/o', 'cong_ket_noi', 'ket_noi_usb','giao_tiep',
      'giao_tiep_mang', 'giao_tiep_mang_-_communications','cong_giao_tiep__port'
])

# Nhóm Expansion
df = combine_columns(df, 'Khe Cắm', [
    'kieu_khe_m.2_ho_tro', 'kieu_khe_m.2_ho_tr','kieu_khe_m.2_ho_tro',
    'loai_khe_m2','kieu_khe_m2_ho_tro','kieu_khe_m2_ho_tr', 'loai_khe_m.2', 'khe_cam_mo_rong', 'khe_mo_rong'
])
# Nhóm Screen
df = combine_columns(df, 'Màn Hình', [
    'man_hinh', 'kich_thuoc_man_hinh', 'man_hinh_-_monitor', 'man_hinh_hien_thi', 'man_hin','man_hinh__monitor'
])

# Nhóm Accessories
df = combine_columns(df, 'Phụ Kiện Đi Kèm', [
    'phu_kien', 'phu_kien_kem', 'phu_kien_kem_theo'
])

# Nhóm Upgradeability
df = combine_columns(df, 'Khả Năng Nâng Cấp', [
    'ho_tro_nang_cap', 'kha_nang_nang_cap_trong_tuong_lai', 'kha_nang_nang_cap_ram'
])
# Nhóm Optical Drive
df = combine_columns(df, 'Ổ Đĩa Quang', [
    'o_dia_quang', 'o_dia_quang_(dvd)', 'o_dia_quang_(odd)', 'o_quang','o_cung/_o_dia_quang', 
    'optical_drive','o_cung_o_dia_quang','o_dia_quang_dvd','o_dia_quang_odd'
])

# Nhóm Backlight
df = combine_columns(df, 'Đèn Led', [
    'den_led_tren_may', 'den_ban_phim', 'loai_den_ban_phim'
])

# Nhóm Power
df = combine_columns(df, 'Nguồn', [
    'nguon', 'bo_nguon','cong_suat_bo_sac'
])

# Nhóm Other Features
df = combine_columns(df, 'Tính Năng Khác', [
    'tinh_nang_dac_biet', 'tinh_nang_khac', 'mo_ta_khac', 'other_supports'
])



C:\Users\toanm\AppData\Local\Temp\ipykernel_25464\5598874.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_column_name] = df[existing].bfill(axis=1).iloc[:, 0]
C:\Users\toanm\AppData\Local\Temp\ipykernel_25464\5598874.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_column_name] = df[existing].bfill(axis=1).iloc[:, 0]
C:\Users\toanm\AppData\Local\Temp\ipykernel_25464\5598874.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=

In [18]:
#Đổi tên các cột
df = combine_columns(df, 'Tên Sản Phẩm', ['product_name'])
df = combine_columns(df, 'Giá Bán', ['price'])
df = combine_columns(df, 'Loại Sản Phẩm', ['san_pham'])
df = combine_columns(df, 'Mẫu Mã', ['model'])
df = combine_columns(df, 'Số Cổng Lưu Trữ Tối Đa', ['so_cong_luu_tru_toi_da'])
df = combine_columns(df, 'Kết Nối', ['ket_noi'])
df = combine_columns(df, 'Webcam', ['webcam'])
df = combine_columns(df, 'Camera', ['camera'])
df = combine_columns(df, 'Kết Nối Không Dây', ['ket_noi_khong_day'])
df = combine_columns(df, 'Giao Tiếp Mạng', ['giao_tiep_mang__communications'])
df = combine_columns(df, 'NPU', ['npu'])
df = combine_columns(df, 'Cổng IO', ['cong_io'])
df = combine_columns(df, 'Cổng Xuất Hình', ['cong_xuat_hinh'])
df = combine_columns(df, 'Loa', ['loa'])
df = combine_columns(df, 'Năm Sản Xuất', ['nam_san_xuat'])
df = combine_columns(df, 'Thiết Kế Chất Liệu Vỏ', ['thiet_ke_chat_lieu_vo'])
df = combine_columns(df, 'Hệ Điều Hành Bản Quyền Đi Kèm', ['he_dieu_hanh_ban_quyen_di_kem'])
df = combine_columns(df, 'Phần Mềm Có Sẵn', ['phan_mem_co_san'])
df = combine_columns(df, 'Card Mở Rộng', ['card_mo_rong'])
df = combine_columns(df, 'Unnamed 161', ['unnamed_161'])
df = combine_columns(df, 'Mãy Tính Xách Tay', ['may_tinh_xach_tay'])

In [19]:
# Thay 'Liên hệ' hoặc các giá trị không phải số thành NaN
df['Giá Bán'] = df['Giá Bán'].replace('Liên hệ', None)
df['Giá Bán'] = (
    df['Giá Bán']
    .astype(str)  # Ép về chuỗi
    .str.replace('đ', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.replace(',', '', regex=False)  # nếu có dấu phẩy
)

# Sau đó chuyển về float, nếu không chuyển được thì thành NaN
df['Giá Bán'] = pd.to_numeric(df['Giá Bán'], errors='coerce')

In [20]:
# Xuất dữ liệu đã làm sạch
df.to_csv("../clean_data_train/clean_mega.csv", index=False)